In [152]:
import pandas as pd
import numpy as np
import os 
import requests
import joblib
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy import stats 
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances


In [153]:
schedule = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/NHL_points_projection/files/schedule/nhl-2024-PacificStandardTime.csv')
skaters = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/NHL_points_projection/files/master_copies/forwards_rec_two.csv')
defence = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/NHL_points_projection/files/master_copies/defense_rec_two.csv')
skaters_25 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/NHL_points_projection/files/skaters/skaters_2024.csv')
skaters_active = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/NHL_points_projection/files/master_copies/forwards_rec_2025.csv')
skaters_team = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/NHL_points_projection/files/skaters/skaters_2024.csv')

In [154]:
combined_df = pd.concat([skaters, defence], ignore_index=True)

combined_df = combined_df.drop(columns=(['Unnamed: 0.1', 'Unnamed: 0']))
combined_df


,name,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,medium_danger_goals,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shots_blocked_by_player,assists,position_encoded,position,salary
0,A.J. Greer,1.243590,0.316239,0.081197,0.038462,0.158120,1.961538,0.034188,0.863248,0.008547,...,234,534.529915,0.015513,0.015897,0.282051,0.371795,0.123932,1,F,950000
1,Aaron Downey,0.000000,0.500000,0.250000,0.000000,0.000000,2.750000,0.000000,0.250000,0.250000,...,4,312.500000,0.112500,0.120000,0.500000,0.250000,0.250000,1,F,950000
2,Aaron Gagnon,1.500000,0.052632,0.078947,0.000000,0.078947,0.631579,0.026316,0.921053,0.052632,...,38,502.894737,0.052632,0.052105,0.026316,0.368421,0.052632,1,F,950000
3,Aaron Palushaj,1.220588,0.191176,0.044118,0.000000,0.044118,0.970588,0.029412,1.132353,0.014706,...,68,542.294118,0.031324,0.030000,0.279412,0.367647,0.161765,1,F,950000
4,Aaron Volpatti,1.035088,0.096491,0.043860,0.017544,0.052632,2.254386,0.017544,0.640351,0.008772,...,114,476.307018,0.020000,0.021404,0.219298,0.280702,0.017544,1,F,950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,Zach Werenski,3.500000,0.551440,0.185185,0.032922,0.104938,0.584362,0.104938,3.236626,0.047325,...,486,1403.553498,0.008868,0.008827,0.092593,1.269547,0.436214,1,D,8000000
2916,Zach Whitecloud,2.773279,0.360324,0.068826,0.008097,0.040486,1.668016,0.048583,1.502024,0.012146,...,247,1055.174089,0.011781,0.012105,0.129555,1.655870,0.165992,1,D,6000000
2917,Zachary Hayes,0.666667,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.666667,0.000000,...,3,653.000000,0.176667,0.156667,0.000000,0.333333,0.000000,1,D,950000
2918,Zbynek Michalek,4.892514,0.397313,0.046065,0.005758,0.017274,1.000000,0.030710,1.708253,0.009597,...,521,1267.504798,0.007236,0.007313,0.147793,2.412668,0.159309,1,D,4000000


In [155]:
skaters_active = skaters_active.drop(columns=['Unnamed: 0'])
# Assuming both dataframes share a "name" column
skaters_active = skaters_active.merge(combined_df[['name', 'salary']], on='name', how='left')
skaters_active

,name,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,medium_danger_goals,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shots_blocked_by_player,assists,position_encoded,position,salary
0,A.J. Greer,1.250000,0.514706,0.088235,0.029412,0.205882,2.720588,0.044118,0.985294,0.014706,...,68,572.985294,0.007500,0.007353,0.367647,0.323529,0.161765,1,F,950000.0
1,Aaron Ekblad,4.446429,1.017857,0.053571,0.000000,0.053571,1.607143,0.053571,2.910714,0.000000,...,56,1410.750000,0.010000,0.009821,0.285714,1.142857,0.535714,0,F,8000000.0
2,Aatu Raty,1.190476,0.190476,0.095238,0.000000,0.000000,2.523810,0.047619,0.666667,0.047619,...,21,570.333333,0.023333,0.021905,0.095238,0.190476,0.095238,1,F,950000.0
3,Adam Boqvist,1.033333,0.466667,0.133333,0.066667,0.100000,0.400000,0.066667,1.600000,0.000000,...,30,810.633333,0.019000,0.018667,0.000000,0.500000,0.233333,0,F,950000.0
4,Adam Edstrom,2.333333,0.372549,0.098039,0.019608,0.215686,1.843137,0.019608,0.470588,0.058824,...,51,556.411765,0.008824,0.008824,0.196078,0.666667,0.078431,1,F,950000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,Zachary L'Heureux,1.739130,0.586957,0.086957,0.043478,0.239130,3.217391,0.000000,1.347826,0.043478,...,46,695.130435,0.011304,0.011087,0.260870,0.326087,0.195652,1,F,950000.0
868,Zack Bolduc,1.000000,0.879310,0.206897,0.103448,0.189655,1.362069,0.051724,1.844828,0.051724,...,58,747.793103,0.009310,0.009310,0.189655,0.517241,0.241379,1,F,950000.0
869,Zack MacEwen,0.571429,0.333333,0.095238,0.000000,0.095238,2.333333,0.095238,1.380952,0.000000,...,21,465.666667,0.025238,0.025238,0.285714,0.333333,0.047619,1,F,950000.0
870,Zack Ostapchuk,0.909091,0.295455,0.022727,0.000000,0.068182,1.977273,0.022727,0.613636,0.000000,...,44,557.863636,0.010000,0.009773,0.227273,0.272727,0.068182,1,F,950000.0


In [156]:
skaters_team.head()

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,...,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
0,8482733,2024,Logan Mailloux,MTL,D,other,7,301.0,4.0,0.27,...,0.43,1.55,6.0,15.0,0.00,0.00,0.0,0.0,0.0,0.0
1,8482733,2024,Logan Mailloux,MTL,D,all,7,6806.0,134.0,1.75,...,15.09,18.74,272.0,337.0,0.00,0.00,0.0,0.0,0.0,0.0
2,8482733,2024,Logan Mailloux,MTL,D,5on5,7,6362.0,124.0,1.75,...,10.27,9.10,216.0,225.0,0.24,0.05,10.0,1.0,8.0,1.0
3,8482733,2024,Logan Mailloux,MTL,D,4on5,7,96.0,3.0,0.30,...,0.00,2.72,0.0,18.0,0.00,0.00,0.0,0.0,0.0,0.0
4,8482733,2024,Logan Mailloux,MTL,D,5on4,7,47.0,3.0,0.30,...,1.63,0.92,21.0,4.0,0.00,0.00,0.0,0.0,0.0,0.0


In [157]:
# Initialize an empty list to store DataFrames
pivoted_dfs = []
cols_to_drop = [
    'gameScore',
    'onIce_xGoalsPercentage',
    'offIce_xGoalsPercentage',
    'offIce_corsiPercentage',
    'offIce_fenwickPercentage',
    'iceTimeRank',
    'I_F_xOnGoal',
    'I_F_xGoals',
    'I_F_xRebounds',
    'I_F_xFreeze',
    'I_F_xPlayStopped',
    'I_F_xPlayContinuedInZone',
    'I_F_xPlayContinuedOutsideZone',
    'I_F_flurryAdjustedxGoals',
    'I_F_scoreVenueAdjustedxGoals',
    'I_F_flurryScoreVenueAdjustedxGoals',
    'I_F_freeze',
    'I_F_playStopped',
    'I_F_playContinuedInZone',
    'I_F_playContinuedOutsideZone',
    'I_F_savedShotsOnGoal',
    'I_F_savedUnblockedShotAttempts',
    'penalties',
    'I_F_lowDangerxGoals',
    'I_F_highDangerxGoals',
    'I_F_mediumDangerxGoals',
    'I_F_scoreAdjustedShotsAttempts',
    'I_F_unblockedShotAttempts',
    'I_F_scoreAdjustedUnblockedShotAttempts',
    'I_F_dZoneGiveaways',
    'I_F_xGoalsFromxReboundsOfShots',
    'I_F_xGoalsFromActualReboundsOfShots',
    'I_F_xGoals_with_earned_rebounds',
    'I_F_xGoals_with_earned_rebounds_scoreAdjusted',
    'I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
    'I_F_neutralZoneShiftStarts',
    'I_F_flyShiftStarts',
    'I_F_oZoneShiftEnds',
    'I_F_dZoneShiftEnds',
    'I_F_neutralZoneShiftEnds',
    'I_F_flyShiftEnds',
    'I_F_faceOffsWon',
    'timeOnBench',
    'penalityMinutes',
    'penalityMinutesDrawn',
    'OnIce_F_xOnGoal',
    'OnIce_F_xGoals',
    'OnIce_F_flurryAdjustedxGoals',
    'OnIce_F_scoreVenueAdjustedxGoals',
    'OnIce_F_flurryScoreVenueAdjustedxGoals',
    'OnIce_F_shotsOnGoal',
    'OnIce_F_missedShots',
    'OnIce_F_blockedShotAttempts',
    'OnIce_F_shotAttempts',
    'OnIce_F_goals',
    'OnIce_F_rebounds',
    'OnIce_F_reboundGoals',
    'OnIce_F_lowDangerShots',
    'OnIce_F_mediumDangerShots',
    'OnIce_F_highDangerShots',
    'OnIce_F_lowDangerxGoals',
    'OnIce_F_mediumDangerxGoals',
    'OnIce_F_highDangerxGoals',
    'OnIce_F_lowDangerGoals',
    'OnIce_F_mediumDangerGoals',
    'OnIce_F_highDangerGoals',
    'OnIce_F_scoreAdjustedShotsAttempts',
    'OnIce_F_unblockedShotAttempts',
    'OnIce_F_scoreAdjustedUnblockedShotAttempts',
    'OnIce_F_xGoalsFromxReboundsOfShots',
    'OnIce_F_xGoalsFromActualReboundsOfShots',
    'OnIce_F_reboundxGoals',
    'OnIce_F_xGoals_with_earned_rebounds',
    'OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted',
    'OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
    'OnIce_A_xOnGoal',
    'OnIce_A_xGoals',
    'OnIce_A_flurryAdjustedxGoals',
    'OnIce_A_scoreVenueAdjustedxGoals',
    'OnIce_A_flurryScoreVenueAdjustedxGoals',
    'OnIce_A_shotsOnGoal',
    'OnIce_A_missedShots',
    'OnIce_A_blockedShotAttempts',
    'OnIce_A_shotAttempts',
    'OnIce_A_goals',
    'OnIce_A_rebounds',
    'OnIce_A_reboundGoals',
    'OnIce_A_lowDangerShots',
    'OnIce_A_mediumDangerShots',
    'OnIce_A_highDangerShots',
    'OnIce_A_lowDangerxGoals',
    'OnIce_A_mediumDangerxGoals',
    'OnIce_A_highDangerxGoals',
    'OnIce_A_lowDangerGoals',
    'OnIce_A_mediumDangerGoals',
    'OnIce_A_highDangerGoals',
    'OnIce_A_scoreAdjustedShotsAttempts',
    'OnIce_A_unblockedShotAttempts',
    'OnIce_A_scoreAdjustedUnblockedShotAttempts',
    'OnIce_A_xGoalsFromxReboundsOfShots',
    'OnIce_A_xGoalsFromActualReboundsOfShots',
    'OnIce_A_reboundxGoals',
    'OnIce_A_xGoals_with_earned_rebounds',
    'OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted',
    'OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
    'OffIce_F_xGoals',
    'OffIce_A_xGoals',
    'OffIce_F_shotAttempts',
    'OffIce_A_shotAttempts',
    'xGoalsForAfterShifts',
    'xGoalsAgainstAfterShifts',
    'corsiForAfterShifts',
    'corsiAgainstAfterShifts',
    'fenwickForAfterShifts',
    'fenwickAgainstAfterShifts',
    'OnIce_A_reboundxGoals',
    'OnIce_A_reboundxGoals',
]

# List of situations
situations = ['all']

# Loop through each situation and pivot the DataFrame
for situation in situations:
    # Filter the DataFrame for the specific situation
    situation_df = skaters_team[skaters_team['situation'] == situation]
    
    # Drop specific columns
    situation_df = situation_df.drop(columns=cols_to_drop)

    # Pivot the DataFrame
    pivoted_df = situation_df.pivot_table(index=['name', 'season'], aggfunc='first')
    
    # Flatten the columns
    pivoted_df.columns = [f'{col}' for col in pivoted_df.columns]
    
    # Reset the index
    pivoted_df = pivoted_df.reset_index()
    
    # Append the pivoted DataFrame to the list
    pivoted_dfs.append(pivoted_df)

# Merge all pivoted DataFrames into one master DataFrame
df_all = pivoted_dfs[0]
for pivoted_df in pivoted_dfs[1:]:
    df_all = df_all.merge(pivoted_df, on=['name', 'season'], how='outer')

In [158]:
# add primary and secondary assist cols to make assist col then drop cols
df_all['assists'] = df_all['I_F_primaryAssists'] + df_all['I_F_secondaryAssists']
df_all = df_all.drop(columns=['I_F_primaryAssists', 'I_F_secondaryAssists'])

In [159]:
# Change the values in the 'position' column based on the position
df_all['position'] = np.where(df_all['position'].str.contains('L|C|R'), 'F', df_all['position'])
df_all['position'] = np.where(df_all['position'].str.contains('D'), 'D', df_all['position'])

# Display the updated 'position' column
print(df_all['position'])

0      F
1      D
2      F
3      D
4      F
      ..
867    F
868    F
869    F
870    F
871    F
Name: position, Length: 872, dtype: object


In [160]:
# Use encoding to give the positions a numeric value
le = LabelEncoder()
df_all['position'] = df_all['position'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x) 

# Fit the encoder and transform the 'position' column
df_all['PositionEn_encoded'] = le.fit_transform(df_all['position'])
df_all = df_all.drop(columns=['position'])

In [161]:
# Group the DataFrame by 'name' and 'season'
df_all_grouped = df_all.groupby(by=['name']).agg({
    'team': 'first',
    'season': 'first',  # Keep the first occurrence of 'season'
    'I_F_dZoneShiftStarts': 'sum',  # Sum the 'I_F_dZoneShiftStarts' values
    'I_F_giveaways': 'sum',  # Sum the 'I_F_giveaways' values
    'I_F_goals': 'sum',  # Sum the 'I_F_goals' values
    'I_F_highDangerGoals': 'sum',  # Sum the 'I_F_highDangerGoals' values
    'I_F_highDangerShots': 'sum',  # Sum the 'I_F_highDangerShots' values
    'I_F_hits': 'sum',  # Sum the 'I_F_hits' values
    'I_F_lowDangerGoals': 'sum',  # Sum the 'I_F_lowDangerGoals' values
    'I_F_lowDangerShots': 'sum',  # Sum the 'I_F_lowDangerShots' values
    'I_F_mediumDangerGoals': 'sum',  # Sum the 'I_F_mediumDangerGoals' values
    'I_F_mediumDangerShots': 'sum',  # Sum the 'I_F_mediumDangerShots' values
    'I_F_missedShots': 'sum',  # Sum the 'I_F_missedShots' values
    'I_F_oZoneShiftStarts': 'sum',  # Sum the 'I_F_oZoneShiftStarts' values
    'I_F_penalityMinutes': 'sum',  # Sum the 'I_F_penalityMinutes' values
    'I_F_points': 'sum',  # Sum the 'I_F_points' values
    'I_F_reboundGoals': 'sum',  # Sum the 'I_F_reboundGoals' values
    'I_F_rebounds': 'sum',  # Sum the 'I_F_rebounds' values
    'I_F_shifts': 'sum',  # Sum the 'I_F_shifts' values
    'I_F_shotAttempts': 'sum',  # Sum the 'I_F_shotAttempts' values
    'I_F_shotsOnGoal': 'sum',  # Sum the 'I_F_shotsOnGoal' values
    'I_F_takeaways': 'sum',  # Sum the 'I_F_takeaways' values
    'faceoffsLost': 'sum',  # Sum the 'faceoffsLost' values
    'faceoffsWon': 'sum',  # Sum the 'faceoffsWon' values
    'games_played': 'sum',  # Sum the 'games_played' values
    'icetime': 'sum',  # Sum the 'icetime' values
    'onIce_corsiPercentage': 'sum',  # Sum the 'onIce_corsiPercentage' values
    'onIce_fenwickPercentage': 'sum',  # Sum the 'onIce_fenwickPercentage' values
    'penaltiesDrawn': 'sum',  # Sum the 'penaltiesDrawn' values
    'playerId': 'first',  # Keep the first occurrence of 'playerId'
    'shifts': 'sum',  # Sum the 'shifts' values
    'shotsBlockedByPlayer': 'sum',  # Sum the 'shotsBlockedByPlayer' values
    'situation': 'first',  # Keep the first occurrence of 'situation'
    'assists': 'sum',  # Sum the 'assists' values
    'PositionEn_encoded': 'first'  # Keep the first occurrence of 'PositionEn_encoded'
}).reset_index()

# Convert the grouped DataFrame to a regular DataFrame
df_all_grouped

,name,team,season,I_F_dZoneShiftStarts,I_F_giveaways,I_F_goals,I_F_highDangerGoals,I_F_highDangerShots,I_F_hits,I_F_lowDangerGoals,...,icetime,onIce_corsiPercentage,onIce_fenwickPercentage,penaltiesDrawn,playerId,shifts,shotsBlockedByPlayer,situation,assists,PositionEn_encoded
0,A.J. Greer,FLA,2024,85.0,35.0,6.0,2.0,14.0,185.0,3.0,...,38963.0,0.51,0.50,25.0,8478421,969.0,22.0,all,11.0,1
1,Aaron Ekblad,FLA,2024,249.0,57.0,3.0,0.0,3.0,90.0,3.0,...,79002.0,0.56,0.55,16.0,8477932,1604.0,64.0,all,30.0,0
2,Aatu Raty,VAN,2024,25.0,4.0,2.0,0.0,0.0,53.0,1.0,...,11977.0,0.49,0.46,2.0,8482691,311.0,4.0,all,2.0,1
3,Adam Boqvist,NYI,2024,31.0,14.0,4.0,2.0,3.0,12.0,2.0,...,24319.0,0.57,0.56,0.0,8480871,535.0,15.0,all,7.0,0
4,Adam Edstrom,NYR,2024,119.0,19.0,5.0,1.0,11.0,94.0,1.0,...,28377.0,0.45,0.45,10.0,8481726,690.0,34.0,all,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,Zachary L'Heureux,NSH,2024,80.0,27.0,4.0,2.0,11.0,148.0,0.0,...,31976.0,0.52,0.51,12.0,8482742,744.0,15.0,all,9.0,1
868,Zack Bolduc,STL,2024,58.0,51.0,12.0,6.0,11.0,79.0,3.0,...,43372.0,0.54,0.54,11.0,8482737,957.0,30.0,all,14.0,1
869,Zack MacEwen,OTT,2024,12.0,7.0,2.0,0.0,2.0,49.0,2.0,...,9779.0,0.53,0.53,6.0,8479772,271.0,7.0,all,1.0,1
870,Zack Ostapchuk,SJS,2024,40.0,13.0,1.0,0.0,3.0,87.0,1.0,...,24546.0,0.44,0.43,10.0,8482859,670.0,12.0,all,3.0,1


In [162]:
# Create a dictionary for renaming columns
rename_dict = {
    'team': 'team',
    'playerId': 'player_id',
    'I_F_dZoneShiftStarts': 'd_zone_shift_starts',
    'I_F_giveaways': 'giveaways',
    'I_F_goals': 'goals',
    'I_F_highDangerGoals': 'high_danger_goals',
    'I_F_highDangerShots': 'high_danger_shots',
    'I_F_hits': 'hits',
    'I_F_lowDangerGoals': 'low_danger_goals',
    'I_F_lowDangerShots': 'low_danger_shots',
    'I_F_mediumDangerGoals': 'medium_danger_goals',
    'I_F_mediumDangerShots': 'medium_danger_shots',
    'I_F_missedShots': 'missed_shots',
    'I_F_oZoneShiftStarts': 'o_zone_shift_starts',
    'I_F_penalityMinutes': 'penalty_minutes',
    'I_F_points': 'points',
    'I_F_reboundGoals': 'rebound_goals',
    'I_F_rebounds': 'rebounds',
    'I_F_shifts': 'shifts',
    'I_F_shotAttempts': 'shot_attempts',
    'I_F_shotsOnGoal': 'shots_on_goal',
    'I_F_takeaways': 'takeaways',
    'faceoffsLost': 'faceoffs_lost',
    'faceoffsWon': 'faceoffs_won',
    'games_played': 'games_played',
    'icetime': 'icetime',
    'onIce_corsiPercentage': 'on_ice_corsi_percentage',
    'onIce_fenwickPercentage': 'on_ice_fenwick_percentage',
    'penaltiesDrawn': 'penalties_drawn',
    'shotsBlockedByPlayer': 'shots_blocked_by_player',
    'PositionEn_encoded': 'position_encoded'
}

# Rename the columns
df_all_grouped = df_all_grouped.rename(columns=rename_dict)

# Display the resulting DataFrame
skater_teams = df_all_grouped.drop(columns=['situation','player_id', 'season'])
skater_teams

,name,team,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,faceoffs_won,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shifts,shots_blocked_by_player,assists,position_encoded
0,A.J. Greer,FLA,85.0,35.0,6.0,2.0,14.0,185.0,3.0,67.0,...,12.0,68,38963.0,0.51,0.50,25.0,969.0,22.0,11.0,1
1,Aaron Ekblad,FLA,249.0,57.0,3.0,0.0,3.0,90.0,3.0,163.0,...,0.0,56,79002.0,0.56,0.55,16.0,1604.0,64.0,30.0,0
2,Aatu Raty,VAN,25.0,4.0,2.0,0.0,0.0,53.0,1.0,14.0,...,102.0,21,11977.0,0.49,0.46,2.0,311.0,4.0,2.0,1
3,Adam Boqvist,NYI,31.0,14.0,4.0,2.0,3.0,12.0,2.0,48.0,...,0.0,30,24319.0,0.57,0.56,0.0,535.0,15.0,7.0,0
4,Adam Edstrom,NYR,119.0,19.0,5.0,1.0,11.0,94.0,1.0,24.0,...,0.0,51,28377.0,0.45,0.45,10.0,690.0,34.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,Zachary L'Heureux,NSH,80.0,27.0,4.0,2.0,11.0,148.0,0.0,62.0,...,0.0,46,31976.0,0.52,0.51,12.0,744.0,15.0,9.0,1
868,Zack Bolduc,STL,58.0,51.0,12.0,6.0,11.0,79.0,3.0,107.0,...,5.0,58,43372.0,0.54,0.54,11.0,957.0,30.0,14.0,1
869,Zack MacEwen,OTT,12.0,7.0,2.0,0.0,2.0,49.0,2.0,29.0,...,6.0,21,9779.0,0.53,0.53,6.0,271.0,7.0,1.0,1
870,Zack Ostapchuk,SJS,40.0,13.0,1.0,0.0,3.0,87.0,1.0,27.0,...,91.0,44,24546.0,0.44,0.43,10.0,670.0,12.0,3.0,1


In [163]:
skaters_all.head()

,name,team,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shifts,shots_blocked_by_player,assists,position_encoded,salary
0,A.J. Greer,FLA,85.0,35.0,6.0,2.0,14.0,185.0,3.0,67.0,...,68,38963.0,0.51,0.50,25.0,969.0,22.0,11.0,1,950000.0
1,Aaron Ekblad,FLA,249.0,57.0,3.0,0.0,3.0,90.0,3.0,163.0,...,56,79002.0,0.56,0.55,16.0,1604.0,64.0,30.0,0,8000000.0
2,Aatu Raty,VAN,25.0,4.0,2.0,0.0,0.0,53.0,1.0,14.0,...,21,11977.0,0.49,0.46,2.0,311.0,4.0,2.0,1,950000.0
3,Adam Boqvist,NYI,31.0,14.0,4.0,2.0,3.0,12.0,2.0,48.0,...,30,24319.0,0.57,0.56,0.0,535.0,15.0,7.0,0,950000.0
4,Adam Edstrom,NYR,119.0,19.0,5.0,1.0,11.0,94.0,1.0,24.0,...,51,28377.0,0.45,0.45,10.0,690.0,34.0,4.0,1,950000.0


In [164]:
mapping = {
    "BOS": "Boston Bruins",
    "ANA": "Anaheim Ducks",
    "BUF": "Buffalo Sabres",
    "CAR": "Carolina Hurricanes",
    "CBJ": "Columbus Blue Jackets",
    "CGY": "Calgary Flames",
    "CHI": "Chicago Blackhawks",
    "COL": "Colorado Avalanche",
    "DAL": "Dallas Stars",
    "DET": "Detroit Red Wings",
    "EDM": "Edmonton Oilers",
    "FLA": "Florida Panthers",
    "LAK": "Los Angeles Kings",
    "MIN": "Minnesota Wild",
    "MTL": "Montreal Canadiens",
    "NJD": "New Jersey Devils",
    "NSH": "Nashville Predators",
    "NYI": "New York Islanders",
    "NYR": "New York Rangers",
    "OTT": "Ottawa Senators",
    "PHI": "Philadelphia Flyers",
    "PIT": "Pittsburgh Penguins",
    "SEA": "Seattle Kraken",
    "SJS": "San Jose Sharks",
    "STL": "St. Louis Blues",
    "TBL": "Tampa Bay Lightning",
    "TOR": "Toronto Maple Leafs",
    "UTA": "Utah Hockey Club",
    "VAN": "Vancouver Canucks",
    "VGK": "Vegas Golden Knights",
    "WPG": "Winnipeg Jets",
    "WSH": "Washington Capitals"
}

# Apply the mapping to the team column
skaters_all["team"] = skaters_all["team"].map(mapping)

skaters_all.head()

,name,team,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shifts,shots_blocked_by_player,assists,position_encoded,salary
0,A.J. Greer,Florida Panthers,85.0,35.0,6.0,2.0,14.0,185.0,3.0,67.0,...,68,38963.0,0.51,0.50,25.0,969.0,22.0,11.0,1,950000.0
1,Aaron Ekblad,Florida Panthers,249.0,57.0,3.0,0.0,3.0,90.0,3.0,163.0,...,56,79002.0,0.56,0.55,16.0,1604.0,64.0,30.0,0,8000000.0
2,Aatu Raty,Vancouver Canucks,25.0,4.0,2.0,0.0,0.0,53.0,1.0,14.0,...,21,11977.0,0.49,0.46,2.0,311.0,4.0,2.0,1,950000.0
3,Adam Boqvist,New York Islanders,31.0,14.0,4.0,2.0,3.0,12.0,2.0,48.0,...,30,24319.0,0.57,0.56,0.0,535.0,15.0,7.0,0,950000.0
4,Adam Edstrom,New York Rangers,119.0,19.0,5.0,1.0,11.0,94.0,1.0,24.0,...,51,28377.0,0.45,0.45,10.0,690.0,34.0,4.0,1,950000.0


In [165]:
#skaters_all = skater_teams.merge(skaters_active[['name', 'salary']])
skaters_all.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/NHL_points_projection/files/master_copies/player_team_2025.csv')
skaters_all.head()

,name,team,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shifts,shots_blocked_by_player,assists,position_encoded,salary
0,A.J. Greer,Florida Panthers,85.0,35.0,6.0,2.0,14.0,185.0,3.0,67.0,...,68,38963.0,0.51,0.50,25.0,969.0,22.0,11.0,1,950000.0
1,Aaron Ekblad,Florida Panthers,249.0,57.0,3.0,0.0,3.0,90.0,3.0,163.0,...,56,79002.0,0.56,0.55,16.0,1604.0,64.0,30.0,0,8000000.0
2,Aatu Raty,Vancouver Canucks,25.0,4.0,2.0,0.0,0.0,53.0,1.0,14.0,...,21,11977.0,0.49,0.46,2.0,311.0,4.0,2.0,1,950000.0
3,Adam Boqvist,New York Islanders,31.0,14.0,4.0,2.0,3.0,12.0,2.0,48.0,...,30,24319.0,0.57,0.56,0.0,535.0,15.0,7.0,0,950000.0
4,Adam Edstrom,New York Rangers,119.0,19.0,5.0,1.0,11.0,94.0,1.0,24.0,...,51,28377.0,0.45,0.45,10.0,690.0,34.0,4.0,1,950000.0


In [166]:
schedule.head()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,04/10/2024 10:00,O2 Czech Republic,Buffalo Sabres,New Jersey Devils,1 - 4
1,2,1,05/10/2024 07:00,O2 Czech Republic,New Jersey Devils,Buffalo Sabres,3 - 1
2,3,1,08/10/2024 13:30,Climate Pledge Arena,Seattle Kraken,St. Louis Blues,2 - 3
3,4,1,08/10/2024 16:00,Amerant Bank Arena,Florida Panthers,Boston Bruins,6 - 4
4,5,1,08/10/2024 19:00,Delta Center,Utah Hockey Club,Chicago Blackhawks,5 - 2
